In [ ]:
import os
from skimage import io
import matplotlib.pyplot as plt

from skimage.measure import regionprops
from skimage.morphology import label
import numpy as np

In [ ]:
labels_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Ground_Truth'
images_path = r'D:\Institute of Genetics and Cancer\Neurosphere assay\Assess_Segmentation_Data\Images'

# Import test image
image_name = 'D - 03.tif'

img_filename = os.path.join(images_path, image_name)
lbl_filename = os.path.join(labels_path, image_name)
image = io.imread(img_filename)
labels = io.imread(lbl_filename)

# Get labels and plot
labels = label(labels)

fig, ax = plt.subplots(1, 3, figsize=(16, 8))
ax[0].imshow(image, cmap='gray')
ax[1].imshow(labels, cmap='nipy_spectral', interpolation='none')
ax[2].imshow(image, cmap='gray')
ax[2].imshow(labels, cmap='nipy_spectral', alpha=.5, interpolation='none')
fig.tight_layout
plt.show()

In [ ]:
import math

# Define additiinal properties
def circularity(p):
    return 4 * math.pi * p['area'] / (p['perimeter_crofton'] ** 2) # using 'perimeter calculates circularities up to 1.5'
def aspect_ratio(p):
    return p['axis_major_length'] / p['axis_minor_length']

# Get properties as numpy arrays
props = regionprops(labels)
area_array = np.asarray([p['area'] for p in props])
solidity_array = np.asarray([p['solidity'] for p in props])
circularity_array = np.asarray([circularity(p) for p in props])
aspect_ratio_array = np.asarray([aspect_ratio(p) for p in props])

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Object distribution for different shape descriptors', fontsize=16)
ax[0,0].set_title('Area')
ax[0,0].set_ylabel('Number of objects')
ax[0,0].hist(area_array, bins=40)
ax[0,1].set_title('Solidity')
ax[0,1].set_ylabel('Number of objects')
ax[0,1].hist(solidity_array, bins=40)
ax[1,0].set_title('Circularity')
ax[1,0].set_ylabel('Number of objects')
ax[1,0].hist(circularity_array, bins=40)
ax[1,1].set_title('Aspect Ratio')
ax[1,1].set_ylabel('Number of objects')
ax[1,1].hist(aspect_ratio_array, bins=40)
plt.show()

In [ ]:
class_list = []
#bin_intervals=[0,250,625,900,10000]

for i in range(len(area_array)):
    label_id = props[i].label # gets the label id of the current region
    
    if(area_array[i]<500):
        class_mask = np.where(labels == label_id, 1, 0)
    elif(area_array[i]<2000):
        class_mask = np.where(labels == label_id, 2, 0)
        class_mask.shape
    elif(area_array[i]<4000):
        class_mask = np.where(labels == label_id, 3, 0)
    else:
        class_mask = np.where(labels == label_id, 4, 0)
    
    class_list.append(class_mask)

# reconstructs the label image, now containing the eroded labels
class_stack = np.stack(class_list) # creates stack from list of images (numpy arrays)
image_classes = np.max(class_stack, axis = 0) # calculates the maximum projection to get back a 2D, labelled image

In [ ]:
from skimage.color import label2rgb

# define RGB colors from Tableau 10 (default in matplotlib)
t10_blue = [31/255, 119/255, 180/255]
t10_orange = [255/255, 127/255, 14/255]
t10_green = [44/255, 160/255, 44/255]
t10_red = [214/255, 39/255, 40/255]

# list of colors
colors=[t10_blue, t10_orange, t10_green, t10_red]

# create RGB image with color-labels over the original, grayscale image
rgb_labels = label2rgb(
    label=image_classes,
    image=image,
    colors=colors,
    alpha=0.7,
    bg_label=0,
    bg_color=None
    )

# plot original and color-coded images
fig, ax = plt.subplots(1,2, figsize=(16,12))
ax[0].imshow(image, cmap='gray')
ax[1].imshow(rgb_labels)
plt.show()